# INT8 Conv2d (inference): поиск быстрых конфигураций

Этот ноутбук перебирает конфигурации INT8-свёртки (`TritonConv2dINT8`) и сравнивает их по скорости и точности с FP32/FP16 эталоном. Квантизация предполагается готовой: вход `x_q` и веса `w_q` — симметричные int8 (scale из `act_scale` и `weight_scale`), аккумуляция int32, де-квант в FP32 + bias. Backward не реализован. Все замеры на CUDA, с прогревом.

**Метрики, которые считаются и выводятся:**
- `time_ms` — среднее время forward INT8-свёртки, мс.
- (Если есть) `torch_time_ms` — время референса (FP32/FP16 Conv2d).
- `speedup_vs_ref` или аналогичный столбец — отношение времени референса к INT8; >1 — INT8 быстрее.
- Ошибки относительно референса:
  - `mae` — средняя абсолютная ошибка.
  - `max` — максимальное абсолютное отклонение.
  - `rel_l2` — относительная L2-норма ошибки.
- Конфигурация задачи: размеры ядра/входа/батча, Cin/Cout, stride/padding/dilation.
- Конфигурация Triton блоков: `BLOCK_M`, `BLOCK_N`, `BLOCK_K`, `NUM_WARPS`, `NUM_STAGES` (для img2col_int8, GEMM int8, col2img_int32).


In [1]:
import sys, pathlib
sys.path.insert(0, str(pathlib.Path().resolve().parent))
print('sys.path[0]=', sys.path[0])

sys.path[0]= /home/manzhura/ITMO/EDLM/conv2d-img2col-gemm


In [2]:
import torch
import itertools
from typing import List, Dict
import torch.nn as nn
import torch.nn.functional as F
import time
import itertools
import pandas as pd
from tqdm import tqdm

In [3]:
from conv_gemm.baseline_layers.triton_conv2d_int8 import TritonConv2dINT8

In [4]:
device = "cuda"
assert torch.cuda.is_available(), "Нужен CUDA для бенчмарка"
torch.manual_seed(0)
torch.cuda.manual_seed(0)

In [5]:
def bench_forward(layer: nn.Module,
                  x: torch.Tensor,
                  warmup: int = 5,
                  iters: int = 50) -> float:

    layer.eval()
    with torch.no_grad():
        # warmup
        for _ in range(warmup):
            _ = layer(x)
        torch.cuda.synchronize()

        # timed runs
        start = torch.cuda.Event(enable_timing=True)
        end = torch.cuda.Event(enable_timing=True)
        start.record()
        for _ in range(iters):
            _ = layer(x)
        end.record()
        torch.cuda.synchronize()

        elapsed_ms = start.elapsed_time(end)  
        return elapsed_ms / iters

In [6]:
def symmetric_scale(x_fp: torch.Tensor, eps: float = 1e-8) -> torch.Tensor:

    x_max = x_fp.abs().max()
    s = x_max / 127.0 + eps
    return s

#   search ranges

In [7]:
def make_config_grid() -> List[Dict[str, int]]:

    batch_sizes = [1, 4, 8, 16]
    in_channels = [1, 3, 8, 16, 32]
    out_channels = [3, 8, 16, 32, 64]
    image_sizes = [32, 64, 128, 256, 512]  
    kernels = [1, 3, 5, 7, 9, 11]           
    strides = [1, 2]

    cfgs = []
    for N, Cin, Cout, H, K, S in itertools.product(
        batch_sizes, in_channels, out_channels, image_sizes, kernels, strides
    ):
        if Cin > Cout:
            continue

        if H >= 256 and K >= 9:
            continue  

        if H == 512 and K >= 7:
            continue  
        if H == 512 and N >= 16:
            continue

        P = K // 2 
        cfgs.append({
            "N": N,
            "Cin": Cin,
            "Cout": Cout,
            "H": H,
            "K": K,
            "S": S,
            "P": P,
        })
    return cfgs


### Что будет посчитано в гриде INT8

Перебираются комбинации (Kh/Kw, Cin/Cout, H/W/B, stride/padding/dilation, BLOCK_*). Для каждой:
- Замер `time_ms` INT8 forward.
- Сравнение с референсом: `mae`, `max`, `rel_l2` (и `speedup_vs_ref`, если референс меряется).
- Параметры конфигурации записываются в DataFrame `df_int8`.
Эта таблица позволит отобрать быстрые и точные варианты.


In [11]:
def run_int8_conv_benchmark(
    warmup: int = 5,
    iters: int = 50,
) -> pd.DataFrame:

    rows: List[Dict[str, float]] = []
    cfgs = make_config_grid()
    total = len(cfgs)


    for idx, cfg in enumerate(cfgs, start=1):
        N   = cfg["N"]
        Cin = cfg["Cin"]
        Cout = cfg["Cout"]
        H   = cfg["H"]
        K   = cfg["K"]
        S   = cfg["S"]
        P   = cfg["P"]

        try:
            
            x_fp16 = torch.randn(N, Cin, H, H, device=device, dtype=torch.float16)
            # torch fp16
            conv_fp = nn.Conv2d(
                in_channels=Cin,
                out_channels=Cout,
                kernel_size=K,
                stride=S,
                padding=P,
                bias=True,
            ).to(device=device, dtype=torch.float16)

            # квантование весов и активаций
            with torch.no_grad():
                act_scale = symmetric_scale(x_fp16.float())
                w_fp32 = conv_fp.weight.detach().float()
                w_scale = symmetric_scale(w_fp32)
                w_q = torch.clamp(torch.round(w_fp32 / w_scale), -128, 127).to(torch.int8)

            # Triton INT8
            conv_int8 = TritonConv2dINT8(
                in_channels=Cin,
                out_channels=Cout,
                kernel_size=K,
                stride=S,
                padding=P,
                dilation=1,
                bias=True,
            ).to(device)

            with torch.no_grad():
                conv_int8.load_quant_params(
                    w_q=w_q,
                    w_scale=w_scale.to(device),
                    act_scale=act_scale.to(device),
                    bias=conv_fp.bias.detach().to(torch.float32),
                )
                x_q = conv_int8.quantize_input(x_fp16)

                # ошибка
                y_ref = conv_fp(x_fp16).float()
                y_int8 = conv_int8(x_q).float()
                diff = (y_ref - y_int8).abs()
                mae = diff.mean().item()
                max_err = diff.max().item()

            # бенч
            t_torch_ms = bench_forward(conv_fp, x_fp16, warmup=warmup, iters=iters)
            t_triton_ms = bench_forward(conv_int8, x_q, warmup=warmup, iters=iters)
            speedup = t_torch_ms / t_triton_ms if t_triton_ms > 0 else float("nan")

            print(
                f"[{idx}/{total}] "
                f"N={N}, Cin={Cin}, Cout={Cout}, H={H}, K={K}, S={S}, P={P} | "
                f"Torch={t_torch_ms:.4f} ms, Triton={t_triton_ms:.4f} ms | "
                f"Speedup={speedup:.4f} | MAE={mae:.5f}"
            )

            rows.append({
                "N": N,
                "Cin": Cin,
                "Cout": Cout,
                "H": H,
                "K": K,
                "S": S,
                "P": P,
                "t_torch_ms": t_torch_ms,
                "t_triton_ms": t_triton_ms,
                "speedup": speedup,
                "mae": mae,
                "max_err": max_err,
                "shape": f"{N} / {Cin}→{Cout} / {H} / {K} / {S} / {P}",
            })

        except RuntimeError as e:
            if "out of memory" in str(e).lower():
                print(
                    f"[{idx}/{total}] OOM: "
                    f"N={N}, Cin={Cin}, Cout={Cout}, H={H}, K={K}, S={S}, P={P} — skip"
                )
                torch.cuda.empty_cache()
                continue
            else:
                raise

    df = pd.DataFrame(rows)
    print("\n>>> Benchmark completed.")
    return df


In [12]:
df_int8 = run_int8_conv_benchmark(warmup=5, iters=50)


>>> Starting INT8 benchmark: 3686 configurations

[1/3686] N=1, Cin=1, Cout=3, H=32, K=1, S=1, P=0 | Torch=0.0276 ms, Triton=0.2711 ms | Speedup=0.1020 | MAE=0.00215
[2/3686] N=1, Cin=1, Cout=3, H=32, K=1, S=2, P=0 | Torch=0.0310 ms, Triton=0.2817 ms | Speedup=0.1102 | MAE=0.00310
[3/3686] N=1, Cin=1, Cout=3, H=32, K=3, S=1, P=1 | Torch=0.0352 ms, Triton=0.2441 ms | Speedup=0.1440 | MAE=0.00523
[4/3686] N=1, Cin=1, Cout=3, H=32, K=3, S=2, P=1 | Torch=0.0232 ms, Triton=0.2357 ms | Speedup=0.0983 | MAE=0.00421
[5/3686] N=1, Cin=1, Cout=3, H=32, K=5, S=1, P=2 | Torch=0.0262 ms, Triton=0.2453 ms | Speedup=0.1069 | MAE=0.00429
[6/3686] N=1, Cin=1, Cout=3, H=32, K=5, S=2, P=2 | Torch=0.0261 ms, Triton=0.2361 ms | Speedup=0.1106 | MAE=0.00354
[7/3686] N=1, Cin=1, Cout=3, H=32, K=7, S=1, P=3 | Torch=0.0256 ms, Triton=0.2348 ms | Speedup=0.1089 | MAE=0.00395
[8/3686] N=1, Cin=1, Cout=3, H=32, K=7, S=2, P=3 | Torch=0.0256 ms, Triton=0.2334 ms | Speedup=0.1096 | MAE=0.00344
[9/3686] N=1, Cin=1, 

In [ ]:
df_int8

### Отбор лучших INT8 конфигураций

Ниже сортируем результаты по времени/скорости (и опционально фильтруем по ошибке). Сохраняйте строки, где:
- `speedup_vs_ref > 1` (INT8 быстрее),
- Ошибка в допустимых пределах для inference (обычно `mae` ~1e-3–1e-2 в зависимости от диапазонов).
Эти варианты можно рекомендовать для конкретных форматов входа/выхода.


In [21]:
df = df_int8.copy()

df["Shape_info"] = (
    df["N"].astype(str) + "/" +
    df["Cin"].astype(str) + "/" +
    df["H"].astype(str) + "/" +
    df["K"].astype(str)
)

idx = df.groupby("H")["speedup"].idxmax()

best = df.loc[idx].copy()

best = (
    best[["H", "Shape_info", "t_torch_ms", "t_triton_ms", "speedup", "mae", "max_err"]]
    .sort_values("H")
    .set_index("H")
)

best


,Shape_info,t_torch_ms,t_triton_ms,speedup,mae,max_err
H,,,,,,
32,4/16/32/11,0.145446,0.299222,0.486082,0.004362,0.024794
64,4/16/64/5,0.102543,0.263209,0.389589,0.005172,0.031538
128,8/8/128/9,2.238710,6.648256,0.336736,0.005319,0.036072
256,16/16/256/1,0.257820,0.456352,0.564959,0.006160,0.040376
512,8/16/512/1,0.511160,1.029427,0.496548,0.006339,0.042889
